In [6]:
import pandas as pd
import csv
import os
import sklearn

from start import data_path
import extract_laws
import clean_documents
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
docs_df = pd.read_csv(os.path.join(data_path,'doi_exemptions_and_dates.csv'))
doi_df = docs_df[['title', 'text','link', 'possible_laws','term_year', 'term_month', 'finalize_year', 'finalize_month']]
print(len(doi_df))
doi_df[doi_df.title == "South San Antonio ISD"]

900


,title,text,link,possible_laws,term_year,term_month,finalize_year,finalize_month
683,South San Antonio ISD,South San Antonio Independent School District ...,https://www.southsanisd.net//cms/lib/TX0191831...,"[25.0811, 21.003, 25.112, 25.1113, 21.051, 25....",2017.0,NaN,2017.0,April


In [8]:
def make_list(string):
    new_list = [i.replace('[','').replace(']','') for i in string.split(', ')]
    return new_list

doi_df['possible_laws'] = doi_df['possible_laws'].apply(make_list)
doi_df.head()

/Users/kylie/.pyenv/versions/3.7.3/envs/dofis/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,text,link,possible_laws,term_year,term_month,finalize_year,finalize_month
0,Abbott ISD,1 Abbott Independent School District District ...,https://www.abbottisd.org/ourpages/auto/2018/3...,"[25.113, 37.0012, 25.0811, 45.206, 21.003, 21....",2018.0,March,2018.0,February
1,Abernathy ISD,Abernathy ISD District of Innovation Plan – Dr...,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,"[25.113, 37.0012, 28.214, 21.003, 21.057, 25.0...",NaN,NaN,2017.0,January
2,Abilene ISD,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,https://www.abileneisd.org/wp-content/uploads/...,"[25.0811, 21.003, 21.053]",2018.0,NaN,2018.0,April
3,Academy ISD,District of Innovation Plan Developed in coope...,https://4.files.edl.io/1a8f/06/29/18/204245-44...,"[37.0012, 25.0811, 21.102, 21.053, 21.003, 21....",2018.0,NaN,2018.0,June
4,Adrian ISD,Adrian ISD District of Innovation Plan House B...,http://www.adrianisd.net/UserFiles/Servers/Ser...,"[25.0811, 28.214, 25.092, 21.102, 21.401, 21.0...",2018.0,NaN,NaN,NaN


In [9]:
mlb = MultiLabelBinarizer()
plans_onehot = doi_df.join(pd.DataFrame(mlb.fit_transform(doi_df.pop('possible_laws')),
                          columns=mlb.classes_,
                          index=doi_df.index))
plans_onehot.head(10)
len(plans_onehot)

900

In [10]:
drop_cols = [c for c in plans_onehot.columns if c.lower()[:7] == 'unnamed']
plans_onehot = plans_onehot.drop(drop_cols, axis=1)
plans_onehot.head()
len(plans_onehot)

900

In [11]:
plans_onehot.columns = ['reg' + str(i).replace('.', '_') for i in plans_onehot.columns]
plans_onehot = plans_onehot.drop(['reg'], axis=1)
plans_onehot = plans_onehot.rename(index=str, columns= {'regtitle':'district', 
                                                        'reglink': 'link',
                                                        'regtext': 'text',
                                                        'regp_innovation':'p_doi',
                                                        'regterm_year' : 'term_year',
                                                       'regterm_month': 'term_month',
                                                       'regfinalize_year': 'finalize_year',
                                                       'regfinalize_month': 'finalize_month'})
plans_onehot.head(10)

,district,text,link,term_year,term_month,finalize_year,finalize_month,reg11_1511,reg11_162,reg11_164,...,reg45_207,reg45_208,reg45_209,reg46_001,reg48_005,reg51_403,reg55_0811,reg74_24,reg75_6,reg97_1001
0,Abbott ISD,1 Abbott Independent School District District ...,https://www.abbottisd.org/ourpages/auto/2018/3...,2018.0,March,2018.0,February,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,Abernathy ISD District of Innovation Plan – Dr...,https://1.cdn.edl.io/epGD4mZXjWUcPl8yA7IhlcDol...,NaN,NaN,2017.0,January,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,https://www.abileneisd.org/wp-content/uploads/...,2018.0,NaN,2018.0,April,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,District of Innovation Plan Developed in coope...,https://4.files.edl.io/1a8f/06/29/18/204245-44...,2018.0,NaN,2018.0,June,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,Adrian ISD District of Innovation Plan House B...,http://www.adrianisd.net/UserFiles/Servers/Ser...,2018.0,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Agua Dulce ISD,Microsoft Word - DOI 2017-2022.docx Agua Dulce...,https://tx02206063.schoolwires.net/cms/lib/TX0...,2017.0,NaN,2017.0,June,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Alamo Heights ISD,1 | P a g e Mission Statement The Alamo Height...,http://www.ahisd.net/UserFiles/Servers/Server_...,2016.0,NaN,2016.0,December,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Albany ISD,Microsoft Word - innovation_plan.docx Albany I...,http://www.albanyisd.net/uploads/4/4/4/1/44419...,2017.0,June,2017.0,March,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Aldine ISD,Aldine District of Innovation Plan Approved by...,http://insidealdine.com/wp-content/uploads/201...,2017.0,NaN,2017.0,May,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Aledo ISD,ALEDO Independent School District A Past to Re...,https://docs.google.com/viewerng/viewer?url=ht...,NaN,NaN,2019.0,February,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
cols = [c for c in plans_onehot.columns if c.lower()[:3] == 'reg']
plans_onehot['total'] = plans_onehot[cols].sum(axis = 1)
print(len(plans_onehot))
# drop documents with missing regulations
#plans_onehot = plans_onehot[plans_onehot.total > 0]
print(len(plans_onehot))

900
900


In [13]:
# replace values
plans_onehot['reg25_0811'] = np.where((plans_onehot.reg25_081 == 1), 1, plans_onehot.reg25_0811)

In [14]:
plans_onehot.to_csv((os.path.join(data_path, 'doi_final_wtext.csv')))
plans_onehot = plans_onehot.drop(['text'], axis = 1)
plans_onehot.to_csv((os.path.join(data_path, 'doi_final.csv')))

In [15]:
df = pd.read_csv((os.path.join(data_path, 'doi_final.csv')))

In [16]:
plans_onehot.reg25_0811.mean()

0.9811111111111112

In [17]:
df.reg25_0811.mean()

0.9811111111111112